In [1]:
TYPE_CLUSTER = ["popularity_type","quality_of_song","type_of_running", "quantity_of_fade", "song_style", "estimated_region"]
NUM_CLUSTER = [6, 4, 4, 3, 5, 10]

FEATURES = [
  ["artist_hotttnesss", "artist_familiarity","song_hotttnesss"],
  ["variability", "song_hotttnesss"],
  ["duration", "tempo"],
  ["fadiness"],
  ["density", "tempo"],
  ["artist_latitude", "artist_longitude"]]

NAMES = [
  ["Average","Popular", "Undefined",'One-hit Wonder','Superstar','Underdog Song'],#singer_type
  ["Average", "Complex", "Unpopular", "Popular"],
  ["Jogging", "Cardio", "Marathon", "Sprint"], #type_of_running 
  ["Long", "Short", "Average"], #fading
  ["Rock/Hip-Hop", "Virtuous", "Dub/Instrumental", "Pop", "Ambient"], #density 
  ["South America","Easten America ","Europe","Middle East","Oceania","North, Central America","North Europe", "Africa","Western America","No data"]] 

GENERAL_QUERY = """ 
  SELECT track_id, artist_location, artist_latitude, artist_longitude,
  duration, tempo, density, fadiness
  FROM songs
 """

QUERY_HOTNESS = """ 
  SELECT track_id,
  artist_familiarity, artist_hotttnesss, song_hotttnesss, variability
  FROM songs WHERE artist_familiarity!=0 AND artist_hotttnesss !=0 AND song_hotttnesss!=0
 """

In [2]:
# PIPELINE --> 1. DATA INGESTION
from os.path import abspath
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,when
from pyspark.sql.types import IntegerType
  
spark = SparkSession.builder.appName("Milion Songs Dataset").config("spark.sql.warehouse.dir", abspath('/user/hive/warehouse/songs')).enableHiveSupport().getOrCreate()

def load_spark_dataset(QUERY_SQL):
  spark_dataset = spark.sql(QUERY_SQL) 
  return  spark_dataset

# DATA INGESTION: LOAD data and extraxt training and testing dataframe
spark_dataset = load_spark_dataset(GENERAL_QUERY)
spark_dataset_hotnesses = load_spark_dataset(QUERY_HOTNESS)

# DATA PREPARATION: TURN "NaN" values into 0 values (to insert them into clustering model)
spark_dataset = spark_dataset.withColumn("artist_longitude", when(col("artist_longitude")=="NaN", 0).otherwise(col("artist_longitude")))
spark_dataset = spark_dataset.withColumn("artist_latitude", when(col("artist_latitude")=="NaN", 0).otherwise(col("artist_latitude")))

# DATA PREPARATION: Add column "features" for the clustering algorithm (one column for the features selected)
assembler_c1 = VectorAssembler(inputCols=FEATURES[0],outputCol='features_C1')
assembler_c2 = VectorAssembler(inputCols=FEATURES[1],outputCol='features_C2')
assembler_stages_hotness = [assembler_c1, assembler_c2]

assembler_c3 = VectorAssembler(inputCols=FEATURES[2],outputCol='features_C3')
assembler_c4 = VectorAssembler(inputCols=FEATURES[3],outputCol='features_C4')
assembler_c5 = VectorAssembler(inputCols=FEATURES[4],outputCol='features_C5')
assembler_c6 = VectorAssembler(inputCols=FEATURES[5],outputCol='features_C6')
assembler_stages = [assembler_c3, assembler_c4,assembler_c5,assembler_c6]

In [3]:
# PIPELINE --> 2. CLUSTERING (GaussianMixture)
from pyspark.ml import Pipeline
from pyspark.ml.clustering import GaussianMixture

# DEFINE MODEL: Gaussian Mixture Model (GMM) 
gmm_c1 = GaussianMixture(k=NUM_CLUSTER[0] , featuresCol="features_C1", predictionCol=TYPE_CLUSTER[0],probabilityCol='probability_' + TYPE_CLUSTER[0], seed=1)
gmm_c2 = GaussianMixture(k=NUM_CLUSTER[1] , featuresCol="features_C2", predictionCol=TYPE_CLUSTER[1],probabilityCol='probability_' + TYPE_CLUSTER[1], seed=1)
clustering_stages_hotness = [gmm_c1,gmm_c2]

gmm_c3 = GaussianMixture(k=NUM_CLUSTER[2] , featuresCol="features_C3", predictionCol=TYPE_CLUSTER[2],probabilityCol='probability_' + TYPE_CLUSTER[2], seed=1)
gmm_c4 = GaussianMixture(k=NUM_CLUSTER[3] , featuresCol="features_C4", predictionCol=TYPE_CLUSTER[3],probabilityCol='probability_' + TYPE_CLUSTER[3], seed=1)
gmm_c5 = GaussianMixture(k=NUM_CLUSTER[4] , featuresCol="features_C5", predictionCol=TYPE_CLUSTER[4],probabilityCol='probability_' + TYPE_CLUSTER[4], seed=1)
gmm_c6 = GaussianMixture(k=NUM_CLUSTER[5] , featuresCol="features_C6", predictionCol=TYPE_CLUSTER[5],probabilityCol='probability_' + TYPE_CLUSTER[5], seed=1)
clustering_stages = [gmm_c3,gmm_c4, gmm_c5, gmm_c6]

# TRAINIG MODEL
pipeline_hotnesses = Pipeline(stages = assembler_stages_hotness + clustering_stages_hotness)
pipeline = Pipeline(stages = assembler_stages + clustering_stages)

model_hotness = pipeline_hotnesses.fit(spark_dataset_hotnesses)
model = pipeline.fit(spark_dataset) 
models = [model_hotness, model]

# PREDICT CLUSTERS
song_labelled_hotnesses = model_hotness.transform(spark_dataset_hotnesses)
song_labelled_general  = model.transform(spark_dataset)
song_labelled = song_labelled_general.join(song_labelled_hotnesses, "track_id")

# DISPLAY OUTCOMES (clustering models)
index_cluster = 0
for item in models:
  for stage in item.stages:
     if(isinstance(stage, VectorAssembler) != True):        
        print("--------------------CLUSTER (type:{0})----------------------------------".format(TYPE_CLUSTER[index_cluster]))
        print("NUMER OF CLUSTER: {0} \nCLUSTERING_FEATURES: {1}".format(NUM_CLUSTER[index_cluster], FEATURES[index_cluster]))

        # Centroid
        centroid = stage.gaussiansDF.select(col("mean").alias("Centroid")).show(truncate=False)

        # Count occurrances
        print("Number of occurrences for each cluster")
        song_labelled.groupBy(TYPE_CLUSTER[index_cluster]).count().orderBy(TYPE_CLUSTER[index_cluster]).show()
        
        index_cluster += 1

--------------------CLUSTER (type:popularity_type)----------------------------------
NUMER OF CLUSTER: 6 
CLUSTERING_FEATURES: ['artist_hotttnesss', 'artist_familiarity', 'song_hotttnesss']
+------------------------------------------------------------+
Centroid |
+------------------------------------------------------------+
[0.39135896998722464,0.5624392497976437,0.39787900221466876]|
[0.5120741118667708,0.7450475504804166,0.5827457570991089] |
[0.46535445530366537,0.5159862271980233,0.48632497455631496]|
[0.3806816890629545,0.5865059832034507,0.5485806672080243] |
[0.6002322504364334,0.7797170462668456,0.5826541846155584] |
[0.3793616278019142,0.5535662047454347,0.2476287707900453] |
+------------------------------------------------------------+

Number of occurrences for each cluster
+---------------+-----+
popularity_type|count|
+---------------+-----+
 0| 1158|
 1| 1103|
 2| 94|
 3| 648|
 4| 233|
 5| 871|
+---------------+-----+

--------------------CLUSTER (type:quality_of_song)----------------------------------
NUMER OF CLUSTER: 4 
CLUSTERING_FEATURES: ['variability', 'song_hotttnesss']
+----------------------------------------+
Centroid |
+----------------------------------------+
[9.596081575128029,0.37035644177498145] |
[18.453620861519287,0.43702852626099903]|
[10.262833146410625,0.24633380397828641]|
[9.37889793144278,0.5669414828519269] |
+----------------------------------------+

Number of occurrences for each cluster
+---------------+-----+
quality_of_song|count|
+---------------+-----+
 0| 885|
 1| 108|
 2| 865|
 3| 2249|
+---------------+-----+

--------------------CLUSTER (type:type_of_running)----------------------------------
NUMER OF CLUSTER: 4 
CLUSTERING_FEATURES: ['duration', 'tempo']
+---------------------------------------+
Centroid |
+---------------------------------------+
[307.89397387860413,108.89701343006104]|
[213.13836039811073,102.34565577480188]|
[638.6998734998449,104.96325726803053] |
[212.87873270613127,140.3743234448078] |
+---------------------------------------+

Number of occurrences for each cluster
+---------------+-----+
type_of_running|count|
+---------------+-----+
 0| 578|
 1| 1394|
 2| 31|
 3| 2104|
+---------------+-----+

--------------------CLUSTER (type:quantity_of_fade)----------------------------------
NUMER OF CLUSTER: 3 
CLUSTERING_FEATURES: ['fadiness']
+--------------------+
Centroid |
+--------------------+
[28.926292573743943]|
[6.296260633271516] |
[12.891620170413983]|
+--------------------+

Number of occurrences for each cluster
+----------------+-----+
quantity_of_fade|count|
+----------------+-----+
 0| 80|
 1| 2813|
 2| 1214|
+----------------+-----+

--------------------CLUSTER (type:song_style)----------------------------------
NUMER OF CLUSTER: 5 
CLUSTERING_FEATURES: ['density', 'tempo']
+--------------------------------------+
Centroid |
+--------------------------------------+
[3.002965188076811,136.963965074149] |
[4.127451244042262,122.63181301360639]|
[3.7191579491383906,93.89837044351478]|
[3.4773580669448383,131.3536245447299]|
[3.492527609130206,130.18124663847206]|
+--------------------------------------+

Number of occurrences for each cluster
+----------+-----+
song_style|count|
+----------+-----+
 0| 390|
 1| 352|
 2| 1072|
 3| 1946|
 4| 347|
+----------+-----+

--------------------CLUSTER (type:estimated_region)----------------------------------
NUMER OF CLUSTER: 10 
CLUSTERING_FEATURES: ['artist_latitude', 'artist_longitude']
+-----------------------------------------------+
Centroid |
+-----------------------------------------------+
[-27.579352479737185,-51.358149122717116] |
[37.67952593942138,-83.4542263507263] |
[51.08104892520479,7.297690479278477] |
[28.25571692112419,6.798515855281528] |
[-33.21487666504934,144.99957839123448] |
[32.70644894273785,-92.15939493109387] |
[52.26146059983134,-0.7639104741622583] |
[1.7448225178698638,29.528408497511464] |
[35.604951736172865,-119.62273107531048] |
[3.3713469802863856E-10,-6.260042073508

In [4]:
#PIPELINE ---> 3. CLUSTERS LABELLING 
from pyspark.sql.functions import lit,udf
from pyspark.sql.types import FloatType

# Mapper --> Go deep into the probability array
map_array = udf(lambda prob,i: float(prob[i]), FloatType())

for i in range(len(TYPE_CLUSTER)):
  print("\n",TYPE_CLUSTER[i])
  
  # Create "confidence column": probability of the predicted cluster
  song_labelled = song_labelled.withColumn("confidence_"+TYPE_CLUSTER[i],map_array(col("probability_"+TYPE_CLUSTER[i]), lit(col(TYPE_CLUSTER[i]))))

  # Labelling 
  for cluster, name in enumerate(NAMES[i]):
    song_labelled = song_labelled.withColumn(TYPE_CLUSTER[i], when(col(TYPE_CLUSTER[i]) == cluster, name).otherwise(col(TYPE_CLUSTER[i])))
    print("LABELLED --> cluster {0} = {1}".format(cluster, name))

popularity_type
LABELLED --> cluster 0 = Average
LABELLED --> cluster 1 = Popular
LABELLED --> cluster 2 = Undefined
LABELLED --> cluster 3 = One-hit Wonder
LABELLED --> cluster 4 = Superstar
LABELLED --> cluster 5 = Underdog Song

 quality_of_song
LABELLED --> cluster 0 = Average
LABELLED --> cluster 1 = Complex
LABELLED --> cluster 2 = Unpopular
LABELLED --> cluster 3 = Popular

 type_of_running
LABELLED --> cluster 0 = Jogging
LABELLED --> cluster 1 = Cardio
LABELLED --> cluster 2 = Marathon
LABELLED --> cluster 3 = Sprint

 quantity_of_fade
LABELLED --> cluster 0 = Long
LABELLED --> cluster 1 = Short
LABELLED --> cluster 2 = Average

 song_style
LABELLED --> cluster 0 = Rock/Hip-Hop
LABELLED --> cluster 1 = Virtuous
LABELLED --> cluster 2 = Dub/Instrumental
LABELLED --> cluster 3 = Pop
LABELLED --> cluster 4 = Ambient

 estimated_region
LABELLED --> cluster 0 = South America
LABELLED --> cluster 1 = Easten America 
LABELLED --> cluster 2 = Europe
LABELLED --> cluster 3 = Middle East
LABELLED --> cluster 4 = Oceania
LABELLED --> cluster 5 = North, Central America
LABELLED --> cluster 6 = North Europe
LABELLED --> cluster 7 = Africa
LABELLED --> cluster 8 = Western America
LABELLED --> cluster 9 = No data

In [5]:
# PIPELINE --> 4. Display raw outcomes of model(table way)

# Build string for querying
confidence_labels = ["confidence_"+ item for item in TYPE_CLUSTER]
feature_labels = []
for type_cluster in FEATURES:
  for feature in type_cluster:
    if feature not in feature_labels:
      feature_labels.append(feature)

# Display
songs = song_labelled.select(feature_labels + TYPE_CLUSTER + confidence_labels)
display(songs)

artist_hotttnesss,artist_familiarity,song_hotttnesss,variability,duration,tempo,fadiness,density,artist_latitude,artist_longitude,popularity_type,quality_of_song,type_of_running,quantity_of_fade,song_style,estimated_region,confidence_popularity_type,confidence_quality_of_song,confidence_type_of_running,confidence_quantity_of_fade,confidence_song_style,confidence_estimated_region
0.5666228875581625,0.9379647503973342,0.685642423804993,10,275.3824,155.038,4.05740000000003,3.021253355334255,0.0,0.0,Popular,Popular,Sprint,Short,Rock/Hip-Hop,No data,0.93947047,0.9940802,0.9121802,0.8821467,0.48187315,1.0
0.40902774457009566,0.58400755313893,0.37598430150044215,7,135.18322,121.371,4.53922,3.9280023067951775,29.18752,-82.14039,Average,Average,Sprint,Short,Virtuous,"North, Central America",0.89266664,0.6323276,0.58188075,0.87567014,0.49349067,0.88058984
0.5124556425769484,0.8063641247652552,0.21508031850922793,7,253.67465,87.033,3.0676500000000146,3.9775357924018024,0.0,0.0,Underdog Song,Unpopular,Cardio,Short,Dub/Instrumental,No data,0.8668233,0.8799447,0.6029878,0.89179075,0.8329793,1.0
0.6017297557100237,0.8216973167394691,0.7155021691230083,10,203.59791,178.339,9.333910000000003,2.5786119317236604,0.0,0.0,Popular,Popular,Sprint,Short,Ambient,No data,0.77559495,0.9949004,0.99315524,0.7019136,0.7064056,1.0
0.4724954766649901,0.658016591872584,0.34580223396532944,8,170.84036,138.471,10.062359999999984,3.482783576433578,0.0,0.0,Average,Average,Sprint,Short,Pop,No data,0.65142417,0.7159334,0.8419952,0.6485843,0.5833124,1.0
0.38010920821880884,0.4420305132891076,0.3027240513818609,8,179.06893,111.199,9.501929999999987,2.647025366153693,38.8991,-77.029,Average,Average,Cardio,Short,Pop,Easten America,0.6820597,0.6402286,0.558064,0.69040996,0.45026106,0.9879655
0.4260093004837602,0.595645308494379,0.21204540548371908,9,162.35057,146.474,14.568569999999994,4.293178644214184,0.0,0.0,Underdog Song,Unpopular,Sprint,Average,Pop,No data,0.82125676,0.89422625,0.92755,0.77495575,0.7230485,1.0
0.34936636110900054,0.5889320858529523,0.26586104921065007,9,320.31302,120.032,4.8410200000000145,2.7348248285380343,0.0,0.0,Underdog Song,Unpopular,Jogging,Short,Pop,No data,0.7887402,0.7746611,0.39107782,0.8709395,0.37360683,1.0
0.5121972430113413,0.5771581238137459,0.38293509007501925,10,233.32526,151.051,5.2822599999999795,3.6515549152286386,0.0,0.0,Undefined,Average,Sprint,Short,Pop,No data,0.49581197,0.655262,0.92806745,0.8629934,0.66925114,1.0
0.3604457658067229,0.5856831498303938,0.37952332442218817,8,289.64526,137.746,9.444259999999986,4.194786408726316,18.11526,-77.27348,Average,Average,Sprint,Short,Pop,"North, Central America",0.5925545,0.63555765,0.7142116,0.6944118,0.5963888,0.98672724


In [6]:
COLUMNS = ["artist_location", "estimated_region"] + FEATURES[5]
TO_FILTER = "-"
#display(song_labelled.select(COLUMNS).filter(song_labelled.estimated_region == TO_FILTER))

In [7]:
# PIPELINE --> 5. Display outcomes (graphical way)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "DejaVu Serif"

df = songs.toPandas()

fig, axes = plt.subplots(3,2,figsize=(20, 18))
sns.set_style("whitegrid")

#Plot a scatterplot for each CLUSTER
popolarity = sns.scatterplot(x= df[FEATURES[0][0]], y=df[FEATURES[0][2]], size = df[FEATURES[0][1]], hue=df[TYPE_CLUSTER[0]], ax=axes[0][0])
popolarity.set_title(TYPE_CLUSTER[0].upper().replace("_", " "), fontsize=25, pad=10)

quality = sns.scatterplot(x= df[FEATURES[1][0]], y=df[FEATURES[1][1]], hue=df[TYPE_CLUSTER[1]], ax=axes[0][1])
quality.set_title(TYPE_CLUSTER[1].upper().replace("_", " "), fontsize=25, pad=10)

running = sns.scatterplot(x= df[FEATURES[2][0]], y=df[FEATURES[2][1]], hue=df[TYPE_CLUSTER[2]], ax=axes[1][0])
running.set_title(TYPE_CLUSTER[2].upper().replace("_", " "), fontsize=25, pad=10)

fading = sns.scatterplot(x= np.linspace(0, 10, num=len(df[FEATURES[3][0]])), y=df[FEATURES[3][0]], hue=df[TYPE_CLUSTER[3]], ax=axes[1][1])
fading.set_title(TYPE_CLUSTER[3].upper().replace("_", " "), fontsize=25, pad=10)

genre = sns.scatterplot(x= df[FEATURES[4][1]], y=df[FEATURES[4][0]], hue=df[TYPE_CLUSTER[4]], ax=axes[2][0])
genre.set_title(TYPE_CLUSTER[4].upper().replace("_", " "), fontsize=25, pad=10)

region = sns.scatterplot(x= df[FEATURES[5][1]], y=df[FEATURES[5][0]], hue=df[TYPE_CLUSTER[5]], ax=axes[2][1])
region.set_title(TYPE_CLUSTER[5].upper().replace("_", " "), fontsize=25, pad=10)

#fig.suptitle("Clusters", fontsize=60, fontweight="regular",x = 0.5)
plt.tight_layout(pad = 3, rect=[0, 0, 1, 0.96]) #(left, bottom, right, top)
plt.show()

In [8]:
#PIPELINE --> 6. Show goodness (confidence of our clusters)

df = songs.toPandas()
plt.rcParams["font.family"] = "DejaVu Serif"
fig, axes = plt.subplots(nrows = 6, figsize=(20, 32))
sns.set_style("whitegrid")
sns.set(font='DejaVu Sans')

col,row = 0,-1
# Plot boxplots
for i, cluster in enumerate(TYPE_CLUSTER):
  bx = sns.boxplot(x = df["confidence_" + cluster], y = df[cluster],ax = axes[i])
  
  bx.set_title(cluster.upper().replace("_", " "), fontsize=30, pad=10)
  bx.set_xlabel("Confidence",fontsize = 20)
  bx.set_ylabel("CLUSTERS",fontsize = 30, color = "tomato",labelpad =30)
  bx.tick_params(axis = "x",labelsize = 17)
  bx.tick_params(axis = "y",labelsize = 20)
  bx.set_xlim(0.2,1)
  
#fig.suptitle("Confidence of the clusters", fontsize=40, fontweight="bold",x = 0.6)
plt.tight_layout(pad = 3, rect=[0, 0, 1, 0.96]) #(left, bottom, right, top)
plt.show()

In [9]:
# PIPELINE --> 7. Save outcomes
clusters = song_labelled.select(["track_id"]+ TYPE_CLUSTER + confidence_labels)
clusters.write.mode('overwrite').format("parquet").saveAsTable("clusters_")
display(clusters)

track_id,popularity_type,quality_of_song,type_of_running,quantity_of_fade,song_style,estimated_region,confidence_popularity_type,confidence_quality_of_song,confidence_type_of_running,confidence_quantity_of_fade,confidence_song_style,confidence_estimated_region
TRBCKSF128F932FA70,Popular,Popular,Sprint,Short,Rock/Hip-Hop,No data,0.93947047,0.9940802,0.9121802,0.8821467,0.48187315,1.0
TRBCLGH12903CCA1D0,Average,Average,Sprint,Short,Virtuous,"North, Central America",0.89266664,0.6323276,0.58188075,0.87567014,0.49349067,0.88058984
TRBCMHI128F148B17F,Underdog Song,Unpopular,Cardio,Short,Dub/Instrumental,No data,0.8668233,0.8799447,0.6029878,0.89179075,0.8329793,1.0
TRBCMJT128F14B12CC,Popular,Popular,Sprint,Short,Ambient,No data,0.77559495,0.9949004,0.99315524,0.7019136,0.7064056,1.0
TRBCMNO128F92D58E5,Average,Average,Sprint,Short,Pop,No data,0.65142417,0.7159334,0.8419952,0.6485843,0.5833124,1.0
TRBCNAJ128F934266F,Average,Average,Cardio,Short,Pop,Easten America,0.6820597,0.6402286,0.558064,0.69040996,0.45026106,0.9879655
TRBCNSM128F933D958,Underdog Song,Unpopular,Sprint,Average,Pop,No data,0.82125676,0.89422625,0.92755,0.77495575,0.7230485,1.0
TRBCNUB12903CA79C7,Underdog Song,Unpopular,Jogging,Short,Pop,No data,0.7887402,0.7746611,0.39107782,0.8709395,0.37360683,1.0
TRBCOBZ128F428005D,Undefined,Average,Sprint,Short,Pop,No data,0.49581197,0.655262,0.92806745,0.8629934,0.66925114,1.0
TRBCOKA128F4280D91,Average,Average,Sprint,Short,Pop,"North, Central America",0.5925545,0.63555765,0.7142116,0.6944118,0.5963888,0.98672724
